##### Pandas Script Converter

##### Importações

In [104]:
import pandas as pd
import pdfplumber
from IPython.display import display
from openpyxl import load_workbook
from openpyxl.styles import Font

##### Caminho de entrada e saída

In [105]:
# Caminho do seu arquivo PDF
pdf_path = r"C:\Users\matheus_meissner\Desktop\github\pandas_script_converter\pandas_script_converter\tests\Script Irmão do Jorel.pdf"
# Caminho de saída do arquivo Excel
output_path = r"C:\Users\matheus_meissner\Desktop\github\pandas_script_converter\pandas_script_converter\tests\resultados\teste3.xlsx"

##### Extrair dados de arquivos pdf

In [106]:
# Abra o PDF e extraia o texto
def extract_pdf_to_text(pdf_path):
    text_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text_data.append(page.extract_text())
    print("Texto extraído do PDF:")
    for page in text_data:
        print(page)
    return text_data

text_data = extract_pdf_to_text(pdf_path)

Texto extraído do PDF:
INSTITUTO FEDERAL DE MATO GROSSO
DISCIPLINA: LÍNGUA INGLESA
Professora: Micaela Coelho
E-mail: micaela.coelho@alf.ifmt.edu.br
WhatsApp: (34) 9 9952-1582
SCRIPT – IRMÃO DO JOREL
Sobre Irmão do Jorel
Irmão do Jorel é uma série brasileira de desenho animado que foi criada em 2014. Ela
foi a primeira animação original do Cartoon Network produzida no Brasil e na
América Latina. Imrão do Jorel conta a história de uma família fora do comum, que
tem três filhos: Nico (o filho mais velho), Jorel (o filho do meio) e o irmão do Jorel
(filho mais novo). Estranhamente, o nome do filho mais novo nunca é revelado;
apesar disso, é ele quem protagoniza a história. O nome da série faz referência ao fato
de que Jorel parece ser o filho preferido – o mais bonito e inteligente; por isso, seu
irmão mais novo vive sendo comparado, ofuscado por ele e se metendo em encrencas.
Episódio 1 da 1ª temporada
SCRIPT
Jorel: Soy dumpling accelerates to the final lap with his soy retropopulsor… su

##### Transforma texto em dataframe

In [107]:
# Transforme o texto em um DataFrame do Pandas
def convert_text_to_dataframe(text_data):
    data = []
    for page in text_data:
        for line in page.split('\n'):
            data.append(line.split('\t'))
    
    # Converta os dados em um DataFrame do Pandas
    df = pd.DataFrame(data)
    print("\nDataFrame inicial convertido do texto extraído:")
    display(df.head(10))  # Mostra as primeiras 10 linhas para visualização
    return df

df = convert_text_to_dataframe(text_data)


DataFrame inicial convertido do texto extraído:


,0
0,INSTITUTO FEDERAL DE MATO GROSSO
1,DISCIPLINA: LÍNGUA INGLESA
2,Professora: Micaela Coelho
3,E-mail: micaela.coelho@alf.ifmt.edu.br
4,WhatsApp: (34) 9 9952-1582
5,SCRIPT – IRMÃO DO JOREL
6,Sobre Irmão do Jorel
7,Irmão do Jorel é uma série brasileira de desen...
8,foi a primeira animação original do Cartoon Ne...
9,América Latina. Imrão do Jorel conta a históri...


##### Remove linhas desnecessárias

In [108]:
# Função para remover as linhas desnecessárias
def remove_unnecessary_rows(df, start_row, end_row):
    # Remove as linhas de start_row até end_row (não inclusivo)
    df_cleaned = df.drop(df.index[start_row:end_row])
    
    # Verifica se a primeira linha contém apenas "0" e, se sim, remove-a
    if df_cleaned.iloc[0].isna().all() or (df_cleaned.iloc[0] == "0").all():
        df_cleaned = df_cleaned.drop(df_cleaned.index[0])
    
    # Redefine o índice sem adicionar a coluna de índice extra
    df_cleaned = df_cleaned.reset_index(drop=True)

    print("\nDataFrame após remoção das linhas desnecessárias e redefinição do índice:")
    display(df_cleaned.head(10))  # Mostra as primeiras 10 linhas para visualização
    return df_cleaned

df_cleaned = remove_unnecessary_rows(df, 0, 17)


DataFrame após remoção das linhas desnecessárias e redefinição do índice:


,0
0,Jorel: Soy dumpling accelerates to the final l...
1,cauliflower dumpling!
2,"Mom: Stop playing with the food, honey! After ..."
3,whenever you want.
4,Jorel: You want me to play with the food after...
5,Mãe: No! I…
6,Jorel: Can I at least talk with the food? Like...
7,"Daddy: No, you’re the one!"
8,"Mom: Edson, can you stop talking to the food i..."
9,"Daddy: I’m not talking to the food, Danusa!"


##### Renomeia primeira linha

In [109]:
# Função para renomear o cabeçalho da célula "A1"
def rename_first_cell(df, new_value):
    """
    Renomeia o cabeçalho da primeira coluna (célula A1) no DataFrame.

    Args:
    df (DataFrame): O DataFrame a ser atualizado.
    new_value (str): O novo valor para a célula "A1".

    Returns:
    DataFrame: O DataFrame atualizado com o novo valor na célula "A1".
    """
    # Renomeia o cabeçalho da primeira coluna
    if len(df.columns) > 0:
        df.columns = [new_value] + list(df.columns[1:])

    print("\nDataFrame após renomear o cabeçalho da célula A1:")
    display(df.head(10))  # Mostra as primeiras 10 linhas para visualização
    
    return df

# Renomeia o cabeçalho da primeira coluna (célula A1)
df_cleaned = rename_first_cell(df_cleaned, "Characters")


DataFrame após renomear o cabeçalho da célula A1:


,Characters
0,Jorel: Soy dumpling accelerates to the final l...
1,cauliflower dumpling!
2,"Mom: Stop playing with the food, honey! After ..."
3,whenever you want.
4,Jorel: You want me to play with the food after...
5,Mãe: No! I…
6,Jorel: Can I at least talk with the food? Like...
7,"Daddy: No, you’re the one!"
8,"Mom: Edson, can you stop talking to the food i..."
9,"Daddy: I’m not talking to the food, Danusa!"


##### Divide personagem de texto

In [110]:
# Função para dividir personagem e script em duas colunas
def split_character_script(df):
    # Verifica se a coluna 'Characters' existe no DataFrame
    if 'Characters' in df.columns:
        # Converte todos os valores da coluna para strings e substitui NaN por ''
        df['Characters'] = df['Characters'].fillna('').astype(str)
        
        # Inicializa a nova coluna 'Script' vazia
        df['Script'] = ''
        
        # Itera sobre cada linha para dividir o conteúdo
        for i in range(len(df)):
            if ':' in df.at[i, 'Characters']:
                # Divide o conteúdo pelo primeiro ": " e atribui às colunas correspondentes
                character, script = df.at[i, 'Characters'].split(':', 1)
                df.at[i, 'Characters'] = character.strip()
                df.at[i, 'Script'] = script.strip()
            else:
                # Se não houver ":", move o conteúdo para 'Script' e repete o personagem acima
                df.at[i, 'Script'] = df.at[i, 'Characters'].strip()
                df.at[i, 'Characters'] = df.at[i-1, 'Characters'].strip() if i > 0 else ''

    print("\nDataFrame após dividir personagens e scripts com validação adicional:")
    display(df.head(10))  # Mostra as primeiras 10 linhas para visualização
    return df

# Chamada da função
df_cleaned = split_character_script(df_cleaned)


DataFrame após dividir personagens e scripts com validação adicional:


,Characters,Script
0,Jorel,Soy dumpling accelerates to the final lap with...
1,Jorel,cauliflower dumpling!
2,Mom,"Stop playing with the food, honey! After you f..."
3,Mom,whenever you want.
4,Jorel,You want me to play with the food after I eat ...
5,Mãe,No! I…
6,Jorel,Can I at least talk with the food? Like daddy ...
7,Daddy,"No, you’re the one!"
8,Mom,"Edson, can you stop talking to the food in fro..."
9,Daddy,"I’m not talking to the food, Danusa!"


##### Salva dataframe em excel

In [111]:
# Salve o DataFrame em um arquivo Excel
def save_dataframe_to_excel(df, output_path):
    df.to_excel(output_path, index=False, engine='openpyxl')  # Salva sem o índice
    print(f"\nDataFrame salvo em {output_path}")

save_dataframe_to_excel(df_cleaned, output_path)


DataFrame salvo em C:\Users\matheus_meissner\Desktop\github\pandas_script_converter\pandas_script_converter\tests\resultados\teste3.xlsx


##### Ajustar largura das colunas

In [114]:
def adjust_column_width(file_path, column_widths):
    """
    Ajusta a largura das colunas especificadas de um arquivo Excel existente.

    Args:
    file_path (str): O caminho do arquivo Excel existente.
    column_widths (dict): Um dicionário com as larguras das colunas. Exemplo: {'A': 15, 'B': 64}.
    """
    # Carrega o arquivo Excel para ajustar a largura das colunas
    workbook = load_workbook(file_path)
    sheet = workbook.active
    
    # Ajusta a largura das colunas conforme especificado no dicionário column_widths
    for column, width in column_widths.items():
        sheet.column_dimensions[column].width = width
    
    # Salva o arquivo Excel com os ajustes
    workbook.save(file_path)
    print(f"Larguras das colunas ajustadas no arquivo Excel salvo em {file_path}.")

# Ajuste as larguras das colunas no arquivo Excel salvo
adjust_column_width(output_path, {'A': 20, 'B': 80})  
# A: 15 (aprox. 4cm), B: 64 (aprox. 16cm)

Larguras das colunas ajustadas no arquivo Excel salvo em C:\Users\matheus_meissner\Desktop\github\pandas_script_converter\pandas_script_converter\tests\resultados\teste3.xlsx.


##### Padroniza fontes e tamanhos

In [115]:
def standardize_font(file_path, font_name='Calibri', font_size=11):
    """
    Padroniza a fonte e o tamanho das células de todo o arquivo Excel.

    Args:
    file_path (str): O caminho do arquivo Excel existente.
    font_name (str): O nome da fonte a ser aplicada (padrão: 'Calibri').
    font_size (int): O tamanho da fonte a ser aplicado (padrão: 11).
    """
    # Carrega o arquivo Excel
    workbook = load_workbook(file_path)
    sheet = workbook.active
    
    # Define a fonte padrão para todas as células
    font_style = Font(name=font_name, size=font_size)
    
    # Aplica a fonte e o tamanho a todas as células usadas na planilha
    for row in sheet.iter_rows():
        for cell in row:
            if cell.value is not None:  # Aplica apenas às células não vazias
                cell.font = font_style
    
    # Salva o arquivo Excel com as alterações de fonte
    workbook.save(file_path)
    print(f"Fonte padronizada para '{font_name}' tamanho {font_size} no arquivo Excel salvo em {file_path}.")

# Padroniza a fonte para 'Arial' e tamanho 12
standardize_font(output_path, font_name='Arial', font_size=12)

NameError: name 'Font' is not defined

##### Aplica cores

##### Aplica bordas

##### Chama as funções

In [ ]:
# Executa as funções
text_data = extract_pdf_to_text(pdf_path)
df = convert_text_to_dataframe(text_data)

# Remove as linhas 1 a 17 (Python é indexado a partir de 0, por isso usamos 0:17)
df_cleaned = remove_unnecessary_rows(df, 0, 17)

# Renomeia o cabeçalho da primeira coluna (célula A1)
df_cleaned = rename_first_cell(df_cleaned, "Characters")
# Renomeia o cabeçalho da primeira coluna (célula B1)
df_cleaned = rename_first_cell(df_cleaned, "Script")

# Salva o DataFrame limpo em um arquivo Excel
save_dataframe_to_excel(df_cleaned, output_path)

print(f"Dados extraídos e salvos em {output_path}")
print(df_cleaned.head(10))  # Mostra as primeiras 10 linhas para visualização